# 0. 실습 환경 구성

## 1. 실습 패키지 설치

In [1]:
!pip3 install -qU python-dotenv oci oracledb PyPDF2 langchain langchain-community

## 2. 환경 변수 로딩

In [2]:
from dotenv import load_dotenv

load_dotenv(verbose=True, override=True)

True

In [35]:
import os 

GENAI_SERVICE_ENDPOINT = os.environ["GENAI_SERVICE_ENDPOINT"]
COMPARMENT_ID=os.environ["COMPARMENT_ID"]
WALLET_DIR=os.environ["WALLET_DIR"]
WALLET_PASSWORD=os.environ["WALLET_PASSWORD"]
DB_USER_PASSWORD=os.environ["DB_USER_PASSWORD"]
DB_USER =os.environ["DB_USER"]
DB_URL =os.environ["DB_URL"]
DSN =os.environ["DSN"]
TABLE= os.environ["TABLE"]
DBURL= os.environ["DBURL"]

In [36]:
with connection.cursor() as cursor:
    cursor.execute("SELECT sysdate FROM dual")
    result = cursor.fetchone()
    print("Current Database Date:", result[0])

Current Database Date: 2025-03-14 07:14:42


In [4]:
WALLET_DIR , DBURL

('/home/opc/genai-agent/adb_wallet',
 '(description=  (retry_count=3)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=d6aqmjs6.adb.us-chicago-1.oraclecloud.com))(connect_data=(service_name=g13fc7c96b5ee55_agentvs_medium.adb.oraclecloud.com))(security=(ssl_server_dn_match=no)))')

## 데이터베이스 접속 테스트

In [34]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
import oracledb
try:
    connection=oracledb.connect(
         config_dir=WALLET_DIR,
         user=DB_USER,
         password=DB_USER_PASSWORD,
         dsn=DSN,
         wallet_location=WALLET_DIR,
         wallet_password=WALLET_PASSWORD)
    with connection.cursor() as cursor:
        cursor.execute("SELECT sysdate FROM dual")
        result = cursor.fetchone()
        print("Current Database Date:", result[0])
    print("Connection version:", connection.version)
except oracledb.DatabaseError as e:
    print(f"DatabaseError: {e}")

Current Database Date: 2025-03-14 07:14:56
Connection version: 23.7.0.25.3


## OCI Gen AI 기본 설정 테스트

In [7]:
import oci

# Default config file and profil
default_config = oci.config.from_file()



In [9]:
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

oci_llm = ChatOCIGenAI(
    model_id="cohere.command-r-plus",
    service_endpoint=GENAI_SERVICE_ENDPOINT,
    compartment_id=COMPARMENT_ID)

## OCI Gen AI 임베딩 기본 설정 테스트

In [10]:
from langchain_community.embeddings import OCIGenAIEmbeddings

embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=GENAI_SERVICE_ENDPOINT,
    compartment_id=COMPARMENT_ID,
)

In [12]:
len(response), response[:5]

(1024, [-0.0027580261, 0.018997192, 0.002828598, -0.025604248, 0.042877197])

In [23]:
query = "삼차방정식의 근과 계수의 관계"
query_vec = embeddings.embed_query(query)

# load docs


In [27]:
import logging
from langchain_core.documents import Document
from langchain.document_loaders import UnstructuredPowerPointLoader

import os

from langchain_community.document_loaders import CSVLoader

file_path="고등학교 수학개념 - math_high_school_category_202502211455.csv"
loader = CSVLoader(file_path)


In [28]:
docs = loader.load()
docs[1:5]

In [41]:

table_name="superlearning"
table_name

'superlearning'

In [42]:
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores import oraclevs
import oracledb

oracle_vs = OracleVS.from_documents(documents=docs,  
                        embedding=embeddings,
                        client=connection,
                        table_name=table_name , 
                        distance_strategy=DistanceStrategy.COSINE)


In [43]:
query = "삼차방정식의 근과 계수의 관계"
query_vec = embeddings.embed_query(query)

In [47]:
retriever = oracle_vs.as_retriever(search_kwargs={"k": 5})

# 🔹 검색 실행
query = "삼차방정식의 근과 계수의 관계"
documents = retriever.invoke(query)

# 🔹 검색된 문서 출력
for i, doc in enumerate(documents):
    print(f"Document {i+1}: {doc.page_content}")
    #print(f" Source \n {doc.metadata}")

Document 1: id: 1606
name: 삼차방정식의 근과 계수의 관계
Document 2: id: 1412
name: 이차방정식의 근과 계수의 관계
Document 3: id: 1607
name: 세 수를 근으로 하는 삼차방정식
Document 4: id: 3615
name: 삼각함수 사이의 관계
Document 5: id: 4902
name: 삼차방정식의 근의 판별
